In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import config
import helpers
#filename = './Auswertung_Susanne.xlsx'
filename = './wearcheck_Jenbacher S Oil.xlsx'

In [3]:
fields = ['Unique Code','General Description','Oil/Fluid Long Name','Unit Hours','Oil/Fluid Hours'] # auf die benötigten Felder beschränken
xdf = pd.read_excel(filename)[fields]
corr_xdf = xdf.applymap(helpers.corr) # correct the contents
# alle Öle im file ermitteln

In [10]:
corr_xdf;

In [5]:
result = {
    'Zeilen': [],
    'Eindeutige Motoren': [],
    'gültige Motoren': [],
    'Kumulierte Öl Stunden': [],
    'Mittlere Öl Stunden pro gültigem Motor': [],
    }
columns = list(corr_xdf['Oil/Fluid Long Name'].unique())
print(columns)

['Mobil Pegasus 705', 'Jenbacher S Oil 40', 'Chevron HDAX 6500 LFG 40', 'Q8 EL-4825', 'CASTROL DURATEC L', 'Mobil Pegasus 610', 'Mobil Pegasus 710', 'Q8 Mahler GR5 SAE40', 'Jenbacher N Oil 40', 'Mobil Pegasus 605 Ultra 40', 'Q8 Mahler GR8', 'CASTROL DURATEC MX']


In [6]:
for oil_name in columns:
    new_xdf = corr_xdf[corr_xdf['Oil/Fluid Long Name'] == oil_name] # nur die Zeilen mit 'oil_name' ausfiltern
    uniquecodes = new_xdf['Unique Code'].unique() # Alle Unique Codes der Motoren auslesen - nur unterschiedliche codes kommen in die Liste
    No_of_Engines = len(uniquecodes)
    result['Zeilen'].append(new_xdf.shape[0])
    result['Eindeutige Motoren'].append(No_of_Engines)
    if new_xdf.shape[0] == 0: #no enties found
        raise ValueError(f"kein Öl mit Namen '{oil_name}' in '{os.path.basename(filename)}' gefunden.")
    oil_sum = 0
    count_sum = 0
    for u in uniquecodes:
        df = new_xdf[new_xdf['Unique Code'] == u].sort_values(by = ['Unit Hours'],ascending=[True])
        try: #try to calculate oil running hours
            oil_age = df['Unit Hours'].max() - df['Unit Hours'].min() + df.iloc[0]['Oil/Fluid Hours']
            if oil_age == oil_age: #this is a trick to check for NAN
                oil_sum += oil_age
                count_sum += 1
        except Exception:
            pass # do nothing on entry rows with missing parameters.
    result['gültige Motoren'].append(count_sum)
    result['Kumulierte Öl Stunden'].append(int(f"{oil_sum:0.0f}"))
    result['Mittlere Öl Stunden pro gültigem Motor'].append(int(f"{oil_sum / count_sum:0.0f}"))

In [13]:
result[' '] = ['']*len(columns)
result['Datum'] = [pd.Timestamp.now()] + ['']* (len(columns)-1)
result['Datei'] = [os.path.basename(filename)]  + ['']* (len(columns)-1)

result;

{'Zeilen': [411, 480, 7, 9, 59, 120, 50, 90, 2, 43, 35, 4],
 'Eindeutige Motoren': [16, 39, 1, 2, 2, 4, 3, 2, 1, 1, 2, 1],
 'gültige Motoren': [16, 37, 1, 2, 2, 4, 3, 2, 1, 1, 2, 1],
 'Kumulierte Öl Stunden': [155126,
  226693,
  5802,
  3794,
  21324,
  51004,
  17737,
  26467,
  3233,
  14235,
  14917,
  3396],
 'Mittlere Öl Stunden pro gültigem Motor': [9695,
  6127,
  5802,
  1897,
  10662,
  12751,
  5912,
  13234,
  3233,
  14235,
  7458,
  3396],
 ' ': ['', '', '', '', '', '', '', '', '', '', '', ''],
 'Datum': [Timestamp('2022-02-06 16:49:40.668333'),
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 'Datei': ['wearcheck_Jenbacher S Oil.xlsx',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '']}

In [14]:
rdf = pd.DataFrame.from_dict(result, columns=columns, orient='index')
#logging.info(f"Exporting Zusammenfassung to {config.zoutfile}.")
rdf.to_excel(config.zoutfile)
#print(rdf)
#os.startfile(config.zoutfile)

In [ ]:
import pandas as pd
t = {
    'A':[1,2,3,4],
    'B':[1,2,3,4],
    'C':[1,2,3,4],
    'D':[1,2,3,4],
    'E':[1,2,3,4],
    ' ':['','','',''],
    'Datum':['1.1.2122','','',''],
    'Datei':['jhgfjhg','','','']
}
tdf = pd.DataFrame.from_dict(t, columns=['Eins','Zwei','Drei','Vier'], orient='index')
tdf.to_excel(config.zoutfile)